# Setup

In [ ]:
%pip install --quiet langchain openai faiss-cpu tiktoken pypdf PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive

drive.mount("/content/drive")

%cd drive/MyDrive/LLM/ulcerative_colitis

Mounted at /content/drive
/content/drive/MyDrive/LLM/ulcerative_colitis


# Import dependencies

In [2]:
import os, json, logging
import os.path as osp
import re
import pandas as pd
from abc import abstractmethod

from typing import Any, Union, Tuple, Dict, Callable, List, Optional, Literal
from pprint import pprint
from datetime import datetime
from langchain.docstore.document import Document
from chromadb.config import Settings

from langchain import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import VectorStore, FAISS, Chroma, Pinecone
import pinecone
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser, ListOutputParser
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.combine_documents.base import BaseCombineDocumentsChain
from langchain.chains.combine_documents.map_reduce import MapReduceDocumentsChain, _split_list_of_docs, _collapse_docs
from langchain.chains.combine_documents.refine import RefineDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.callbacks.manager import Callbacks
from custom_parsers import DrugOutput, DrugParser

import config
from config import MAIN_DIR, DATA_DIR, ARTIFACT_DIR, DOCUMENT_SOURCE

from shutil import rmtree
from utils import load_single_document, load_documents, convert_json_to_documents, convert_csv_to_documents
import yaml

from pydantic import root_validator

In [3]:
PROJECT = "uc"

with open(osp.join(MAIN_DIR, "auth", "api_keys.json"), "r") as f:
    keys = json.load(f)

OPENAI_KEY = keys["OPENAI_API_KEY"]
SOURCE_DATA = os.path.join(DOCUMENT_SOURCE, PROJECT)
EMBSTORE_DIR = os.path.join(config.EMBSTORE_DIR, PROJECT, "faiss", "text-embedding-ada-002")

EXCLUDE_DICT = {
    "agrawal.pdf": [13, 14, 15, 16, 17, 18],
    "PIIS1542356520300446.pdf": [12, 13, 14, 15, 16, 17, 18],
    "gutjnl-2021-326390R2 CLEAN.pdf": [0, 2, 31, 32, 33, 34, 35, 36,
                                       37, 38, 39, 40, 41, 42, 43, 44, 45]\
                                        + list(range(3, 31)),
    "otad009.pdf": [15, 16],
    "1-s2.0-S2468125321003770-main.pdf": [9],
    "juillerat 2022.pdf": [6, 7, 8],
}

In [4]:
LOGGER = logging.getLogger()

log_path = os.path.join(MAIN_DIR, "log", "logfile.txt")
file_handler = logging.FileHandler(
    filename=log_path)

formatter = logging.Formatter("%(asctime)s:%(levelname)s: %(message)s")
file_handler.setFormatter(formatter)

LOGGER.setLevel(logging.INFO)
LOGGER.addHandler(file_handler)

# User-defined Functions (UDF)

In [12]:
def convert_csv_to_documents(table_info: Dict, concatenate_rows: bool = True) -> List[Document]:
    """Convert a dictionary containing table information into list of Documents

    Args:
        table_info (Dict): Dictionary containing .csv table information

    Returns:
        List[Document]: List of rows inside the table
    """
    assert table_info["mode"] == "table" and table_info["filename"].endswith(".csv")
    rows = load_single_document(os.path.join(MAIN_DIR, table_info["filename"]))
    documents = []
    table_content = table_info["description"] + "\n\n"
    for row in rows:
        if concatenate_rows:
            table_content += row.page_content + "\n\n"
            table_doc = Document(
                page_content=table_content,
                metadata=table_info["metadata"]
            )
        else:
            row_no = row.metadata["row"]
            metadata = {k: v for k, v in table_info["metadata"].items()}
            metadata["row"] = row_no
            metadata["modal"] = table_info["mode"]
            row.page_content = table_info["description"] + ":" + row.page_content
            row.metadata = metadata
            documents.append(row)
            
    if concatenate_rows:
        documents.append(table_doc)
    
    return documents


def generate_vectorstore(
    embeddings: Callable,
    source_directory: Optional[str] = None,
    output_directory: str = "./vectorstore",
    emb_store_type: str = "faiss",
    chunk_size: int = 1000,
    chunk_overlap: int = 250,
    exclude_pages: Optional[Dict] = None,
    pinecone_idx_name: Optional[str] = None,
    additional_docs: Optional[List] = None,
    key_path: Optional[str] = os.path.join(MAIN_DIR, "auth", "api_keys.json"),
) -> VectorStore:
    """Generate New Vector Index Database

    Args:
        source_directory (str): Directory contains source documents
        embeddings (Callable): Function to convert text to vector embeddings
        output_directory (str, optional): Output directory of vector index database. Defaults to "./vectorstore".
        emb_store_type (str, optional): Type of vector index database. Defaults to "faiss".
        chunk_size (int, optional): Maximum size of text chunks (characters) after split. Defaults to 1000.
        chunk_overlap (int, optional): Maximum overlapping window between text chunks. Defaults to 250.
        exclude_pages (Optional[Dict], optional): Dictionary of pages to be excluded from documents. Defaults to None.
        pinecone_idx_name (Optional[str], optional): Name of pinecone index to be created or loaded. Defaults to None.
        additional_docs (Optional[str], optional): Additional Tables, Images or Json to be added to doc list. Defaults to None.
        key_path (Optional[str], optional): Path to file containing API info.
            Defaults to os.path.join(MAIN_DIR, "auth", "api_keys.json").

    Returns:
        Vectorstore: Vector Database
    """

    if os.path.exists(output_directory):
        rmtree(output_directory)
    os.makedirs(output_directory, exist_ok=True)

    if source_directory:
        LOGGER.info(f"Loading documents from {source_directory}")

        documents = load_documents(source_directory, exclude_pages=exclude_pages)
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size, chunk_overlap=chunk_overlap
        )
        texts = text_splitter.split_documents(documents)

        LOGGER.info(f"Loaded {len(documents)} documents from {source_directory}")
        LOGGER.info(
            f"Split into {len(texts)} chunks of text (max. {chunk_size} characters each)"
        )
    else:
        texts = []

    if additional_docs:
        texts.extend(additional_docs)

    LOGGER.info(
        f"Total number of text chunks to create vector index store: {len(texts)}"
    )

    if emb_store_type == "chroma":
        chroma_settings = Settings(
            chroma_db_impl="duckdb+parquet",
            persist_directory=output_directory,
            anonymized_telemetry=False,
        )
        db = Chroma.from_documents(
            texts,
            embeddings,
            persist_directory=output_directory,
            client_settings=chroma_settings,
        )
        db.persist()

    elif emb_store_type == "faiss":
        db = FAISS.from_documents(texts, embedding=embeddings)
        db.save_local(output_directory)
        assert "index.faiss" in os.listdir(
            output_directory
        ) and "index.pkl" in os.listdir(output_directory)

    elif emb_store_type == "pinecone":
        with open(key_path, "r") as f:
            keys = json.loads(f)
        PINECONE_API_KEY = keys["PINECONE_API"]["KEY"]
        PINECONE_ENV = keys["PINECONE_API"]["ENV"]

        pinecone.init(
            api_key=PINECONE_API_KEY,
            environment=PINECONE_ENV,
        )

        if not pinecone_idx_name:
            pinecone_idx_name = "index_{}".format(
                datetime.now().strftime("%d-%m-%Y-%H:%M:%S")
            )

        if pinecone_idx_name not in pinecone.list_indexes():
            db = Pinecone.from_documents(
                texts, embedding=embeddings, index_name=pinecone_idx_name
            )

        else:
            db = Pinecone.from_existing_index(pinecone_idx_name, embeddings)
            db.add_documents(texts)

    LOGGER.info(
        f"Successfully created {emb_store_type} vectorstore at {output_directory}"
    )

    return db
    
def check_documents_token(
    docs: List[Document],
    llm = ChatOpenAI(temperature=0,
                     model_name="gpt-3.5-turbo",
                     openai_api_key=OPENAI_KEY)
    ):
    if not isinstance(docs, List):
        docs = [docs]
    combine_document_chain = StuffDocumentsChain(
        llm_chain=LLMChain(
            llm=llm,
            prompt=PromptTemplate(template="{summaries}",
                                input_variables=["summaries"]),
            verbose=False,
        ),
        verbose=False
    )
    return combine_document_chain.prompt_length(docs)

# Classes

## Custom Chains

In [14]:
class MapReduceDocumentsChainV2(MapReduceDocumentsChain):
    combine_max_tokens: int = 30000
    collapse_max_tokens: int = 5000

    @root_validator()
    def check_maximum_context_length(cls, values: Dict) -> Dict:
        max_token_dict = {
            "gpt-3.5-turbo": 3000,
            "gpt-3.5-turbo-16k": 14000,
            "gpt-4": 7000,
            "gpt-4-32k": 30000
        }
        
        combine_doc_llm_model = values["combine_document_chain"].llm_chain.llm.model_name
        if combine_doc_llm_model in max_token_dict:
            if max_token_dict[combine_doc_llm_model] < values["combine_max_tokens"]:
                values["combine_max_tokens"] = max_token_dict[combine_doc_llm_model]
        
        if values["collapse_document_chain"]:
            collapse_doc_llm_model = values["collapse_document_chain"].llm_chain.llm.model_name
        else:
            collapse_doc_llm_model = values["combine_document_chain"].llm_chain.llm.model_name
        
        if collapse_doc_llm_model in max_token_dict:
            if max_token_dict[collapse_doc_llm_model] < values["collapse_max_tokens"]:
                values["collapse_max_tokens"] = max_token_dict[collapse_doc_llm_model]

        return values

    def combine_docs(
        self,
        docs: List[Document],
        callbacks: Callbacks = None,
        **kwargs: Any,
    ) -> Tuple[str, dict]:
        """Combine documents in a map reduce manner.

        Combine by mapping first chain over all documents, then reducing the results.
        This reducing can be done recursively if needed (if there are many documents).
        """
        results = self.llm_chain.apply(
            # FYI - this is parallelized and so it is fast.
            [{self.document_variable_name: d.page_content, **kwargs} for d in docs],
            callbacks=callbacks,
        )
        return self._process_results(
            results, docs, callbacks=callbacks, **kwargs
        )

    def _process_results(
        self,
        results: List[Dict],
        docs: List[Document],
        callbacks: Callbacks = None,
        **kwargs: Any,
    ) -> Tuple[str, dict]:
        question_result_key = self.llm_chain.output_key
        result_docs = [
            Document(page_content=r[question_result_key], metadata=docs[i].metadata)
            # This uses metadata from the docs, and the textual results from `results`
            for i, r in enumerate(results)
        ]
        length_func = self.combine_document_chain.prompt_length
        num_tokens = length_func(result_docs, **kwargs)

        def _collapse_docs_func(docs: List[Document], **kwargs: Any) -> str:
            return self._collapse_chain.run(
                input_documents=docs, callbacks=callbacks, **kwargs
            )

        collapse_counter = 0
        while num_tokens is not None and num_tokens > self.combine_max_tokens:
            
            # 
            collapse_counter += 1
            if collapse_counter == 2:
                raise Exception("Double Collapse steps. Stop")            
            
            new_result_doc_list = _split_list_of_docs(
                result_docs, length_func, self.collapse_max_tokens, **kwargs
            )
            result_docs = []
            for docs in new_result_doc_list:
                new_doc = _collapse_docs(docs, _collapse_docs_func, **kwargs)
                result_docs.append(new_doc)
            num_tokens = self.combine_document_chain.prompt_length(
                result_docs, **kwargs
            )
        if self.return_intermediate_steps:
            _results = [r[self.llm_chain.output_key] for r in results]
            extra_return_dict = {"intermediate_steps": _results}
        else:
            extra_return_dict = {}
        output = self.combine_document_chain.run(
            input_documents=result_docs, callbacks=callbacks, **kwargs
        )
        return output, extra_return_dict

## Experiment

In [15]:
class BaseExperiment:
    def __init__(
        self,
        llm_type: str = "gpt-3.5-turbo",
        keys_json: str = osp.join(MAIN_DIR, "auth", "api_keys.json"),
        temperature: float = 0,
        max_tokens: int = 512,
        gt: Optional[str] = None,
        verbose: bool = False,
    ):
        self.llm_type = llm_type.lower()
        self.temperature = temperature
        self.max_tokens = max_tokens

        with open(keys_json, "r") as f:
            keys = json.load(f)

        self.openai_key = (
            keys["OPENAI_API_KEY_FOR_GPT4"]
            if self.llm_type == "gpt-4"
            else keys["OPENAI_API_KEY"]
        )
        
        self.ground_truth = self.load_groundtruth(gt) if gt else None    
        self.chain = None
        self.verbose = verbose
        
    @abstractmethod
    def run_test_cases(
        self, test_cases: Union[List[str], str], **kwargs
    ):
        return NotImplementedError
    
    @staticmethod
    def convert_prompt_to_string(
        prompt: Union[PromptTemplate, ChatPromptTemplate]
    ) -> str:
        """Convert Prompt Object to string format

        Args:
            prompt (Union[PromptTemplate, ChatPromptTemplate]): Prompt Template

        Returns:
            str: Prompt String Template
        """
        return prompt.format(**{v: v for v in prompt.input_variables})
    
    def load_groundtruth(self, gt_path: str) -> pd.DataFrame:
        """Load Ground Truth information from .csv file

        Args:
            gt_path (str): Path to Ground Truth file

        Returns:
            pd.DataFrame: DataFrame containing Ground Truth data.
        """
        return pd.read_csv(gt_path, encoding="ISO-8859-1")

class QuestionAnsweringWithIndexSearchExperiment(BaseExperiment):
    """Experiment Module"""

    def __init__(
        self,
        prompt_template: Union[PromptTemplate, ChatPromptTemplate],
        vector_store: str,
        llm_type: str = "gpt-3.5-turbo",
        emb: str = "text-embedding-ada-002",
        keys_json: str = osp.join(MAIN_DIR, "auth", "api_keys.json"),
        temperature: float = 0,
        max_tokens: int = 512,
        gt: Optional[str] = None,
        verbose: bool = False,
        k: int = 4,
        max_tokens_limit: int = 3375,
        reduce_k_below_max_tokens: bool = True,
    ):
        """Initiate Instance for an experiment run

        Args:
            prompt_template (Union[PromptTemplate, ChatPromptTemplate]): Prompt to be feed to LLM
            vector_store (str): Path to Vector Index Database
            llm_type (str, optional): Type of LLM Model. Defaults to "gpt-3.5-turbo".
            emb (str, optional): Type of Embedding Model. Defaults to "text-embedding-ada-002".
            keys_json (str, optional): Path to API Keys. Defaults to osp.join(MAIN_DIR, "auth", "api_keys.json").
            temperature (float, optional): Temperature Settings for LLM model. Lower temperature makes LLM more deterministic
                while higher temperature makes LLM more random. Defaults to 0.
            max_tokens (int, optional): Max_Tokens Settings for LLM model. Defaults to 512.
            gt (Optional[str], optional): Path to Ground Truth file. Defaults to None.
            verbose (bool, optional): Verbose Setting. Defaults to False.
        """

        super(QuestionAnsweringWithIndexSearchExperiment, self).__init__(
            llm_type=llm_type,
            keys_json=keys_json,
            temperature=temperature,
            max_tokens=max_tokens,
            gt=gt,
            verbose=verbose,
        )
        
        if isinstance(prompt_template, ChatPromptTemplate):
            self.llm = ChatOpenAI(
                model_name=self.llm_type,
                temperature=self.temperature,
                max_tokens=self.max_tokens,
                openai_api_key=self.openai_key,
            )
        else:
            self.llm = OpenAI(
                model_name=self.llm_type,
                temperature=self.temperature,
                max_tokens=self.max_tokens,
                openai_api_key=self.openai_key,
            )
            
        self.embedder = OpenAIEmbeddings(model=emb, openai_api_key=self.openai_key)
        try:
            self.load_vectorstore(vector_store)
        except Exception:
            print(
                "Vectorstore invalid. Please load valid vectorstore or create new vectorstore."
            )

        self.k = k
        self.max_tokens_limit = max_tokens_limit
        self.reduce_k_below_max_tokens = reduce_k_below_max_tokens
        self.prompt_template = prompt_template
        self.questions = []
        self.answers = []
        self.sources = []
        self.drug_parser = OutputFixingParser.from_llm(parser= PydanticOutputParser(pydantic_object=DrugOutput),
                                                       llm=ChatOpenAI(model_name="gpt-4",
                                                                      temperature=0,
                                                                      openai_api_key=OPENAI_KEY)
                                                       )
       

    def load_vectorstore(self, vectorstore_path: str):
        """Load Vectorstore from path

        Args:
            vectorstore_path (str): Path to vector database folder.
        """
        assert "index.faiss" in os.listdir(
            vectorstore_path
        ) and "index.pkl" in os.listdir(vectorstore_path), "Invalid Vectorstore"
        self.docsearch = FAISS.load_local(vectorstore_path, self.embedder)
        LOGGER.info("Successfully loaded existing vectorstore from local storage")

    def generate_vectorstore(
        self,
        data_directory: Optional[str] = None,
        output_directory: str = "./vectorstore",
        emb_store_type: str = "faiss",
        chunk_size: int = 1000,
        chunk_overlap: int = 250,
        exclude_pages: Optional[Dict] = None,
        pinecone_idx_name: Optional[str] = None,
        additional_docs: Optional[str] = None,
        key_path: Optional[str] = os.path.join(MAIN_DIR, "auth", "api_keys.json"),
    ):
        """Generate New vectorstore

        Args:
            data_directory (str): Directory contains source documents
            output_directory (str, optional): Output directory of vector index database. Defaults to "./vectorstore".
            emb_store_type (str, optional): Type of vector index database. Defaults to "faiss".
            chunk_size (int, optional): Maximum size of text chunks (characters) after split. Defaults to 1000.
            chunk_overlap (int, optional): Maximum overlapping window between text chunks. Defaults to 250.
            exclude_pages (Optional[Dict], optional): Dictionary of pages to be excluded from documents. Defaults to None.
            pinecone_idx_name (Optional[str], optional): Name of pinecone index to be created or loaded. Defaults to None.
            additional_docs (Optional[str], optional): Additional Tables, Images or Json to be added to doc list. Defaults to None.
            key_path (Optional[str], optional): Path to file containing API info.
                Defaults to os.path.join(MAIN_DIR, "auth", "api_keys.json").
        """
        self.docsearch = generate_vectorstore(
            data_directory=data_directory,
            embedder=self.embedder,
            output_directory=output_directory,
            emb_store_type=emb_store_type,
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            exclude_pages=exclude_pages,
            pinecone_idx_name=pinecone_idx_name,
            additional_docs=additional_docs,
            key_path=key_path,
        )

    def run_test_cases(
        self,
        test_cases: Union[List[str], str],
        only_return_source: bool = False,
        chain_type: Literal["stuff", "refine", "map_reduce", "map_rerank"] = "stuff"
    ):
        """Run and save test cases to memory

        Args:
            test_cases (Union[List[str], str]): List of test queries.
        """
        if isinstance(test_cases, str):
            with open(test_cases, "r", encoding="utf-8-sig") as f:
                test_cases = f.readlines()
            test_cases = [test_case.rstrip() for test_case in test_cases]

        if not self.chain:
            self._create_chain(chain_type = chain_type)

        if only_return_source:
            LOGGER.info("Perform Semantic Search for Source Documents only (No QA).")

        for test_case in test_cases:
            print("Query: {}".format(test_case))
            sources = []  # All sources for 1 single query
            if only_return_source:
                self.questions.append(test_case)
                self.answers.append(None)
                inputs = {"question": test_case}
                source_documents = self.chain._get_docs(inputs)

            else:
                output = self.chain(test_case)
                self.questions.append(output["question"])
                self.answers.append(output["answer"])
                source_documents = output["source_documents"]

            for document in source_documents:
                sources.append(
                    {
                        "title": document.metadata["title"],
                        "filename": document.metadata["source"].split("/")[-1],
                        "page": document.metadata["page"],
                        "text": document.page_content,
                    }
                )

            self.sources.append(sources)

    def reset(self):
        """Reset queries and answers"""
        self.questions = []
        self.answers = []
        self.sources = []
        self.ground_truth = None

    @staticmethod
    def process_source(source: Dict) -> str:
        """_summary_

        Args:
            source (Dict): Source Document Information

        Returns:
            str: Source Document Information in string
        """
        return "\n\n".join([f"{k}: {v}" for k, v in source.items()])

    def save_json(self, output_path: str):
        """Save Output of test case runs to json file

        Args:
            output_path (str): Output Path to json file.
        """
        output_dict = {}
        output_dict["prompt"] = QuestionAnsweringWithIndexSearchExperiment.convert_prompt_to_string(
            self.prompt_template
        )
        output_dict["test_cases"] = []

        for question, answer, source in zip(self.questions, self.answers, self.sources):
            output_dict["test_cases"].append(
                {"question": question, "answer": answer, "sources": source}
            )

        with open(output_path, "w") as f:
            json.dump(output_dict, f)

    def load_json(self, json_path: str, reset: bool = False):
        """Load Queries and Answers from Json file

        Args:
            json_path (str): Path to json output file to load into instance
            reset (bool, optional): If reset, clear queries and answers from memory before loading. Defaults to False.
        """
        if reset:
            self.reset()
        with open(json_path, "r") as f:
            input_dict = json.load(f)
        for test_case in input_dict["test_cases"]:
            self.questions.append(test_case["question"])
            self.answers.append(test_case["answer"])
            self.sources.append(test_case["sources"])

    def write_csv(self, output_csv: str):
        """Write questions and answers to .csv files

        Args:
            output_csv (str): Path to output csv file
        """

        pd_answers = [[], []]
        pd_pros = [[], []]
        pd_cons = [[], []]
        pd_sources = [[], [], [], [], [], []]

        for answer, sources in zip(self.answers, self.sources):
            if answer:
                drugs_info = re.findall(re.compile(r"{[^{}]+}"), answer)
                drugs = []
                for drug in drugs_info:
                    try:
                        drug = self.drug_parser.parse(drug)
                        drugs.append(drug)
                    except Exception:
                        pass
            else:
                drugs = []
                
            pd_answers[0].append(drugs[0].drug_name if len(drugs) > 0 else None)
            pd_answers[1].append(drugs[1].drug_name if len(drugs) > 1 else None)
            pd_pros[0].append(drugs[0].advantages if len(drugs) > 0 else None)
            pd_cons[0].append(drugs[0].disadvantages if len(drugs) > 0 else None)
            pd_pros[1].append(drugs[1].advantages if len(drugs) > 1 else None)
            pd_cons[1].append(drugs[1].disadvantages if len(drugs) > 1 else None)

            for idx, source in enumerate(sources):
                pd_sources[idx].append(QuestionAnsweringWithIndexSearchExperiment.process_source(source))

            if idx + 1 < len(pd_sources):
                for i in range(idx + 1, len(pd_sources)):
                    pd_sources[i].append(None)

        info = {"question": self.questions}

        if self.ground_truth is not None:
            info["gt_rec1"] = self.ground_truth["Recommendation 1"].tolist()
            info["gt_rec2"] = self.ground_truth["Recommendation 2"].tolist()
            info["gt_rec3"] = self.ground_truth["Recommendation 3"].tolist()
            info["gt_avoid"] = self.ground_truth["Drug Avoid"].tolist()
            info["gt_reason"] = self.ground_truth["Reasoning"].tolist()

        info["prompt"] = [
            QuestionAnsweringWithIndexSearchExperiment.convert_prompt_to_string(self.prompt_template)
        ] * len(self.questions)
        info["raw_answer"] = self.answers
        info["answer1"] = pd_answers[0]
        info["pro1"] = pd_pros[0]
        info["cons1"] = pd_cons[0]
        info["answer2"] = pd_answers[1]
        info["pro2"] = pd_pros[1]
        info["cons2"] = pd_cons[1]
        info["source1"] = pd_sources[0]
        info["source2"] = pd_sources[1]
        info["source3"] = pd_sources[2]
        info["source4"] = pd_sources[3]
        info["source5"] = pd_sources[4]
        info["source6"] = pd_sources[5]

        panda_df = pd.DataFrame(info)

        panda_df.to_csv(output_csv, header=True)

    def _create_chain(
        self,
        chain_type: str = "stuff",
        return_source_documents: bool = True,
    ):
        """Initiate QA from Source Chain

        Args:
            chain_type (str, optional): Chain Type. Can be stuff|map_reduce|refine|map_rerank. Defaults to "stuff".
            return_source_documents (bool, optional): Whether to return source documents along side answers. Defaults to True.
        """
        self.chain = RetrievalQAWithSourcesChain.from_chain_type(
            llm=self.llm,
            chain_type=chain_type,
            retriever=self.docsearch.as_retriever(k = self.k),
            return_source_documents=return_source_documents,
            chain_type_kwargs={"prompt": self.prompt_template},
            max_tokens_limit=self.max_tokens_limit,
            reduce_k_below_max_tokens=self.reduce_k_below_max_tokens,
            verbose=self.verbose,
        )
    
class QuestionAnsweringOverDocsExperiment(BaseExperiment):
    """QAOverDocs Base Experiment Module"""

    def __init__(
        self,
        keys_json: str = osp.join(MAIN_DIR, "auth", "api_keys.json"),
        temperature: float = 0,
        gt: Optional[str] = None,
        verbose: bool = False,
    ):
        """QAOverDocs Base Experiment Module

        Args:
            keys_json (str, optional): Path to API Keys. Defaults to osp.join(MAIN_DIR, "auth", "api_keys.json").
            temperature (float, optional): Temperature Settings for LLM model. Defaults to 0.
            gt (Optional[str], optional): Path to Ground Truth file. Defaults to None.
            verbose (bool, optional): Verbose Setting. Defaults to False.
        """
        super(QuestionAnsweringOverDocsExperiment, self).__init__(
            keys_json=keys_json,
            temperature=temperature,
            gt=gt,
            verbose=verbose,
        )

        self.questions = []
        self.answers = []
        self.intermediate_steps = []
        self.prompt_map = {}

        self.drug_parser = DrugParser.from_llm(
            parser=PydanticOutputParser(pydantic_object=DrugOutput),
            llm=ChatOpenAI(
                model_name="gpt-4", temperature=0, openai_api_key=self.openai_key
            ),
        )

    def run_test_cases(
        self,
        test_cases: Union[List[str], str],
        docs: List[Document],
        return_intermediate_steps: bool = False,
    ):
        """Run and save test cases to memory

        Args:
            test_cases (Union[List[str], str]): List of test queries.
            docs (List[Document]): List of input documents
            return_intermediate_steps (bool, optional): Return intermediate steps. Defaults to False.
        """

        if isinstance(test_cases, str):
            with open(test_cases, "r", encoding="utf-8-sig") as f:
                test_cases = f.readlines()
            test_cases = [test_case.rstrip() for test_case in test_cases]

        if not self.chain:
            self._create_chain(return_intermediate_steps=return_intermediate_steps)

        if self.chain._chain_type == "map_reduce_documents_chain":
            no_tokens = sum(
                [
                    self.chain.combine_document_chain.llm_chain.llm.get_num_tokens(
                        doc.page_content
                    )
                    for doc in docs
                ]
            )
            LOGGER.info(f"Number of tokens from all documents: {no_tokens}")

        for test_case in test_cases:
            print("Query: {}".format(test_case))
            output = self.chain({"input_documents": docs, "question": test_case})
            self.questions.append(output["question"])
            self.answers.append(output["output_text"])
            self.intermediate_steps.append(
                output["intermediate_steps"] if return_intermediate_steps else None
            )

    def reset(self):
        """Reset queries and answers"""
        self.questions = []
        self.answers = []
        self.intermediate_steps = []

    @abstractmethod
    def _create_chain(
        self, return_intermediate_steps: bool = False
    ) -> BaseCombineDocumentsChain:
        """Initiate Main Chain for QA

        Args:
            return_intermediate_steps (bool, optional): Return Intermediate Steps. Defaults to False.

        Returns:
            BaseCombineDocumentsChain: QA Chain
        """
        return NotImplementedError

    def save_json(self, output_path: str):
        """Save Output of test case runs to json file

        Args:
            output_path (str): Output Path to json file.
        """
        output_dict = {
            "prompt": {
                prompt_type: BaseExperiment.convert_prompt_to_string(prompt)
                for prompt_type, prompt in self.prompt_map.items()
            },
            "test_cases": [],
        }

        for question, answer, intermediate_steps in zip(
            self.questions, self.answers, self.intermediate_steps
        ):
            output_dict["test_cases"].append(
                {
                    "question": question,
                    "answer": answer,
                    "intermediate_steps": intermediate_steps,
                }
            )

        with open(output_path, "w") as f:
            json.dump(output_dict, f)

    def load_json(self, json_path: str, reset: bool = False):
        """Load Queries and Answers from Json file

        Args:
            json_path (str): Path to json output file to load into instance
            reset (bool, optional): If reset, clear queries and answers from memory before loading. Defaults to False.
        """
        if reset:
            self.reset()
        with open(json_path, "r") as f:
            input_dict = json.load(f)
        for test_case in input_dict["test_cases"]:
            self.questions.append(test_case["question"])
            self.answers.append(test_case["answer"])
            self.intermediate_steps.append(test_case["intermediate_steps"])
        LOGGER.info("Json file loaded successfully into Experiment instance.")

    def write_csv(self, output_csv: str):
        """Write Output to csv file

        Args:
            output_csv (str): Path to csv file
        """
        info = {"question": self.questions}

        if self.ground_truth is not None:
            info["gt_rec1"] = self.ground_truth["Recommendation 1"].tolist()
            info["gt_rec2"] = self.ground_truth["Recommendation 2"].tolist()
            info["gt_rec3"] = self.ground_truth["Recommendation 3"].tolist()
            info["gt_avoid"] = self.ground_truth["Drug Avoid"].tolist()
            info["gt_reason"] = self.ground_truth["Reasoning"].tolist()

        for prompt_type, prompt in self.prompt_map.items():
            info[f"{prompt_type}_prompt"] = [
                BaseExperiment.convert_prompt_to_string(prompt)
            ] * len(self.questions)

        pd_answers = [[], []]
        pd_pros = [[], []]
        pd_cons = [[], []]
        for answer in self.answers:
            if answer:
                try:
                    drugs = self.drug_parser.parse(answer)
                except Exception:
                    raise Exception("Cannot parse answer properly.")
            else:
                drugs = []

            pd_answers[0].append(drugs[0].drug_name if len(drugs) > 0 else None)
            pd_answers[1].append(drugs[1].drug_name if len(drugs) > 1 else None)
            pd_pros[0].append(drugs[0].advantages if len(drugs) > 0 else None)
            pd_cons[0].append(drugs[0].disadvantages if len(drugs) > 0 else None)
            pd_pros[1].append(drugs[1].advantages if len(drugs) > 1 else None)
            pd_cons[1].append(drugs[1].disadvantages if len(drugs) > 1 else None)

        info["raw_answer"] = self.answers
        info["answer1"] = pd_answers[0]
        info["pro1"] = pd_pros[0]
        info["cons1"] = pd_cons[0]
        info["answer2"] = pd_answers[1]
        info["pro2"] = pd_pros[1]
        info["cons2"] = pd_cons[1]

        panda_df = pd.DataFrame(info)

        panda_df.to_csv(output_csv, header=True)


class MapReduceQAOverDocsExperiment(QuestionAnsweringOverDocsExperiment):
    """MapReduce QA Experiment"""

    def __init__(
        self,
        map_prompt: Optional[Union[PromptTemplate, ChatPromptTemplate]],
        combine_prompt: Optional[Union[PromptTemplate, ChatPromptTemplate]],
        collapse_prompt: Optional[Union[PromptTemplate, ChatPromptTemplate]] = None,
        llm_type: str = "gpt-3.5-turbo",
        reduce_llm: Optional[str] = None,
        collapse_llm: Optional[str] = None,
        keys_json: str = osp.join(MAIN_DIR, "auth", "api_keys.json"),
        temperature: float = 0,
        max_gen_tokens: int = 512,
        combine_max_gen_tokens: int = 512,
        collapse_max_gen_tokens: int = 512,
        combine_max_doc_tokens: int = 14000,
        collapse_max_doc_tokens: int = 6000,
        gt: Optional[str] = None,
        verbose: bool = False,
        **kwargs,
    ):
        """MapReduce QA Experiment

        Args:
            map_prompt (Optional[Union[PromptTemplate, ChatPromptTemplate]]): Prompt to perform QA on each document
            combine_prompt (Optional[Union[PromptTemplate, ChatPromptTemplate]]): Prompt to combine all output for final answer
            collapse_prompt (Optional[Union[PromptTemplate, ChatPromptTemplate]], optional): Prompt to answer on small groups of documents. Defaults to None.
            llm_type (str, optional): Base LLM. By default use for map chain. Defaults to "gpt-3.5-turbo".
            reduce_llm (Optional[str], optional): LLM use for final combine step. Defaults to None.
            collapse_llm (Optional[str], optional): LLM use for collapse step. Defaults to None.
            keys_json (str, optional): Path to API keys. Defaults to osp.join(MAIN_DIR, "auth", "api_keys.json").
            temperature (float, optional): Temperature of all LLM models. Defaults to 0.
            max_gen_tokens (int, optional): Max allowed tokens generated for base LLM. Defaults to 512.
            combine_max_gen_tokens (int, optional): Max allowed tokens generated for combine LLM.. Defaults to 512.
            collapse_max_gen_tokens (int, optional): Max allowed tokens generated for collapse LLM.. Defaults to 512.
            combine_max_doc_tokens (int, optional): Maximum combined tokens at final reduce step. Defaults to 14000.
            collapse_max_doc_tokens (int, optional): Maximum number of tokens allowed for each collapse step. Defaults to 6000.
            gt (Optional[str], optional): Path to ground_truth file. Defaults to None.
            verbose (bool, optional): Verbose settings. Defaults to False.
        """

        super(MapReduceQAOverDocsExperiment, self).__init__(
            keys_json=keys_json,
            temperature=temperature,
            gt=gt,
            verbose=verbose,
        )

        if isinstance(map_prompt, ChatPromptTemplate):
            self.llm = ChatOpenAI(
                model_name=llm_type,
                temperature=self.temperature,
                max_tokens=max_gen_tokens,
                openai_api_key=self.openai_key,
            )
        elif isinstance(map_prompt, PromptTemplate):
            self.llm = OpenAI(
                model_name=llm_type,
                temperature=self.temperature,
                max_tokens=max_gen_tokens,
                openai_api_key=self.openai_key,
            )
        else:
            raise ValueError("Incorrect Type of Map Prompt Template")

        reduce_llm = reduce_llm or llm_type
        collapse_llm = collapse_llm or reduce_llm

        if isinstance(combine_prompt, ChatPromptTemplate):
            self.reduce_llm = ChatOpenAI(
                model_name=reduce_llm,
                temperature=self.temperature,
                max_tokens=combine_max_gen_tokens,
                openai_api_key=self.openai_key,
            )

        elif isinstance(combine_prompt, PromptTemplate):
            self.reduce_llm = OpenAI(
                model_name=reduce_llm,
                temperature=self.temperature,
                max_tokens=combine_max_gen_tokens,
                openai_api_key=self.openai_key,
            )
        else:
            raise ValueError("Incorrect Type of Combine Prompt Template")

        self.map_prompt = map_prompt
        self.combine_prompt = combine_prompt
        self.collapse_prompt = collapse_prompt or combine_prompt

        self.prompt_map = {
            "map": map_prompt,
            "combine": combine_prompt,
            "collapse": collapse_prompt,
        }

        if isinstance(self.collapse_prompt, ChatPromptTemplate):
            self.collapse_llm = ChatOpenAI(
                model_name=collapse_llm,
                temperature=self.temperature,
                max_tokens=collapse_max_gen_tokens,
                openai_api_key=self.openai_key,
            )

        elif isinstance(self.collapse_prompt, PromptTemplate):
            self.collapse_llm = OpenAI(
                model_name=collapse_llm,
                temperature=self.temperature,
                max_tokens=collapse_max_gen_tokens,
                openai_api_key=self.openai_key,
            )
        else:
            raise ValueError("Incorrect Type of Collapse Prompt Template")

        self.combine_max_doc_tokens = combine_max_doc_tokens
        self.collapse_max_doc_tokens = collapse_max_doc_tokens

    def _create_chain(self, return_intermediate_steps: bool = False):
        """Initiate QA from Source Chain

        Args:
            return_intermediate_steps (bool, optional): Whether to return intermediate_steps. Defaults to True.
        """

        map_chain = LLMChain(llm=self.llm, prompt=self.map_prompt, verbose=self.verbose)

        reduce_chain = LLMChain(
            llm=self.reduce_llm, prompt=self.combine_prompt, verbose=self.verbose
        )

        combine_document_chain = StuffDocumentsChain(
            llm_chain=reduce_chain,
            document_variable_name="summaries",
            verbose=self.verbose,
        )

        collapse_chain = LLMChain(
            llm=self.collapse_llm, prompt=self.collapse_prompt, verbose=self.verbose
        )

        collapse_document_chain = StuffDocumentsChain(
            llm_chain=collapse_chain,
            document_variable_name="summaries",
            verbose=self.verbose,
        )

        self.chain = MapReduceDocumentsChainV2(
            llm_chain=map_chain,
            combine_document_chain=combine_document_chain,
            collapse_document_chain=collapse_document_chain,
            document_variable_name="context",
            combine_max_tokens=self.combine_max_doc_tokens,
            collapse_max_tokens=self.collapse_max_doc_tokens,
            return_intermediate_steps=return_intermediate_steps,
            return_map_steps=return_intermediate_steps,
            verbose=self.verbose,
        )


class RefineQAOverDocsExperiment(QuestionAnsweringOverDocsExperiment):
    """Refine QA Experiment"""

    def __init__(
        self,
        initial_prompt: Optional[Union[PromptTemplate, ChatPromptTemplate]],
        refine_prompt: Optional[Union[PromptTemplate, ChatPromptTemplate]],
        llm_type: str = "gpt-3.5-turbo",
        refine_llm: Optional[str] = None,
        keys_json: str = osp.join(MAIN_DIR, "auth", "api_keys.json"),
        temperature: float = 0,
        max_gen_tokens: int = 1024,
        gt: Optional[str] = None,
        verbose: bool = False,
        **kwargs,
    ):
        """Refine QA Experiment

        Args:
            initial_prompt (Optional[Union[PromptTemplate, ChatPromptTemplate]]): Prompt for initial answer
            refine_prompt (Optional[Union[PromptTemplate, ChatPromptTemplate]]): Prompt for each refine step.
            llm_type (str, optional): Base LLM. By default use for initial & refine chain. Defaults to "gpt-3.5-turbo".
            refine_llm (Optional[str], optional): LLM for refine chain. In None, assign to base llm type. Defaults to None.
            keys_json (str, optional): Path to API keys file. Defaults to osp.join(MAIN_DIR, "auth", "api_keys.json").
            temperature (float, optional): Temperature of all LLM models. Defaults to 0.
            max_gen_tokens (int, optional): Maximum generated tokens allowed. Defaults to 1024.
            gt (Optional[str], optional): Path to ground truth file. Defaults to None.
            verbose (bool, optional): Verbose settings. Defaults to False.
        """

        super(RefineQAOverDocsExperiment, self).__init__(
            keys_json=keys_json,
            temperature=temperature,
            gt=gt,
            verbose=verbose,
        )

        if isinstance(initial_prompt, ChatPromptTemplate):
            self.llm = ChatOpenAI(
                model_name=llm_type,
                temperature=self.temperature,
                max_tokens=max_gen_tokens,
                openai_api_key=self.openai_key,
            )
        elif isinstance(initial_prompt, PromptTemplate):
            self.llm = OpenAI(
                model_name=llm_type,
                temperature=self.temperature,
                max_tokens=max_gen_tokens,
                openai_api_key=self.openai_key,
            )
        else:
            raise ValueError("Incorrect Type of Map Prompt Template")

        refine_llm = refine_llm or llm_type

        if isinstance(refine_prompt, ChatPromptTemplate):
            self.refine_llm = ChatOpenAI(
                model_name=refine_llm,
                temperature=self.temperature,
                max_tokens=max_gen_tokens,
                openai_api_key=self.openai_key,
            )

        elif isinstance(refine_prompt, PromptTemplate):
            self.refine_llm = OpenAI(
                model_name=refine_llm,
                temperature=self.temperature,
                max_tokens=max_gen_tokens,
                openai_api_key=self.openai_key,
            )

        else:
            raise ValueError("Incorrect Type of Combine Prompt Template")

        self.initial_prompt = initial_prompt
        self.refine_prompt = refine_prompt

        self.prompt_map = {
            "initial": initial_prompt,
            "refine": refine_prompt,
        }

    def _create_chain(self, return_intermediate_steps: bool = False):
        """Initiate QA from Source Chain

        Args:
            return_intermediate_steps (bool, optional): Whether to return intermediate_steps. Defaults to True.
        """

        initial_chain = LLMChain(
            llm=self.llm, prompt=self.initial_prompt, verbose=self.verbose
        )

        refine_chain = LLMChain(
            llm=self.refine_llm, prompt=self.refine_prompt, verbose=self.verbose
        )

        self.chain = RefineDocumentsChain(
            initial_llm_chain=initial_chain,
            refine_llm_chain=refine_chain,
            document_variable_name="context_str",
            initial_response_name="existing_answer",
            return_intermediate_steps=return_intermediate_steps,
            verbose=self.verbose,
        )

# Create Vectorstore

In [ ]:
datastore_paths = [os.path.join(DOCUMENT_SOURCE, PROJECT, file_name) for file_name in os.listdir(os.path.join(DOCUMENT_SOURCE, PROJECT)) if file_name.endswith(".pdf")]
print("Number of documents in datastore:", len(datastore_paths))
for i, path in enumerate(datastore_paths):
    print(f"Index {i + 1}: {path}")

In [ ]:
sample_path = datastore_paths[0]

sample_data = load_single_document(sample_path)
print("Number of pages:", len(sample_data))
sample_page = sample_data[4]
print(str(sample_page.metadata) + "\n")
content = sample_page.page_content
print(f"Text Length: {len(content)}\n")
# content = re.sub(r"\t+", " ", content)
pprint(content[:1000])
metadata = sample_page.metadata
pprint(metadata)

### Ada-Text-Embeddings-2: Text + Tables

In [ ]:
add_docs_path = os.path.join(MAIN_DIR, "data/additional_docs.json")

with open(add_docs_path, "r") as f:
    additional_documents = json.load(f)
    
add_docs = []
for table_info in additional_documents:
    add_docs.extend(convert_csv_to_documents(table_info))
    
add_docs

In [10]:
emb_model = OpenAIEmbeddings(openai_api_key = OPENAI_KEY)
database_path = os.path.join(EMBSTORE_DIR, PROJECT,
                                "faiss", "text-embedding-ada-002", "v2-add")
embstore_type="faiss"
chunk_size = 1000
chunk_overlap = 200

In [11]:
generate_vectorstore(
    embeddings=emb_model,
    source_directory=SOURCE_DATA,
    output_directory=database_path,
    emb_store_type=embstore_type,
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    exclude_pages=EXCLUDE_DICT,
    additional_docs=add_docs)

# Prototypes

## Test Cases

In [7]:
with open(osp.join(DATA_DIR, "queries", "uc.txt"), "r", encoding = "utf-8-sig") as f:
    test_cases = f.readlines()

test_cases = [test_case.rstrip() for test_case in test_cases]
test_cases

['40 year old male with newly diagnosed moderate UC and articular extraintestinal manifestations',
 '70 year old female with newly diagnosed severe UC',
 '35 year old male with known moderate UC with prior exposure to infliximab but has worsening colitis on endoscopy despite compliance',
 '60 year old female with newly diagnosed moderate UC with a background of congestive cardiac failure',
 '38 year old female with newly diagnosed moderate UC and psoriasis',
 '25 year old pregnant woman with severe distal ulcerative colitis',
 '56 year old man with moderate to severe ulcerative colitis and ankylosing spondylitis',
 '38 year old man with severe ulcerative colitis and has lost response to vedolizumab',
 '28 year old woman who has severe extensive ulcerative colitis and has a history of lymphoma which was treated 4 years ago',
 '36 year old woman with moderate ulcerative colitis and multiple sclerosis']

## Experiment 1: Only Text - Normal Prompt Template - GPT4

### Prompt Setup

In [ ]:
### STANDARD PROMPT TEMPLATE
drug_parser = PydanticOutputParser(pydantic_object=DrugOutput)

prompt_template = """Make reference to the context given to assess the scenario. If you do not know the answer. just say that "I don't know", don't try to make up an answer.
You are a physician assistant giving advice on treatment for moderate to severe ulcerative colitis (UC). Perform the following step

ANALYSE the given patient profile based on given query based on one of the following criteria:
- Freshly treated patient or patient under maintenance
- Prior response to Infliximab
- Prior failure to Anti-TNF agents
- Prior failure to Vedolizumab
- Age
- Pregnancy
- Extraintestinale manifestations
- Pouchitis

FINALLY RETURN up to 2 TOP choices of biological drugs given patient profile. Explain the PROS and CONS of the 2 choices.

{summaries}

{format_instructions}

Question: {question}
Answer:
"""

TEST_PROMPT_TEMPLATE_1 = PromptTemplate(
    template = prompt_template,
    input_variables = ["summaries", "question"],
    partial_variables={"format_instructions": drug_parser.get_format_instructions()}
)

print(TEST_PROMPT_TEMPLATE_1.format(summaries = "Summaries", question = "User Query"))

Make reference to the context given to assess the scenario. If you do not know the answer. just say that "I don't know", don't try to make up an answer.
You are a physician assistant giving advice on treatment for moderate to severe ulcerative colitis (UC). Perform the following step

ANALYSE the given patient profile based on given query based on one of the following criteria:
- Freshly treated patient or patient under maintenance
- Prior response to Infliximab
- Prior failure to Anti-TNF agents
- Prior failure to Vedolizumab
- Age
- Pregnancy
- Extraintestinale manifestations
- Pouchitis

FINALLY RETURN up to 2 TOP choices of biological drugs given patient profile. Explain the PROS and CONS of the 2 choices.

Summaries

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
t

### Run Experiments

In [16]:
# Settings
llm_type = "gpt-4"
description = "normal_prompt_1000_200"
emb_store_dir = os.path.join(EMBSTORE_DIR, "v1_1000_50")
max_tokens = 1024
time = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
verbose = True
save_path = osp.join(ARTIFACT_DIR, f"{llm_type}_{description}_{time}")

In [ ]:
# Create and run experiment
exp1 = QuestionAnsweringWithIndexSearchExperiment(
    prompt_template = TEST_PROMPT_TEMPLATE_1,
    vector_store = emb_store_dir,
    llm_type = llm_type,
    max_tokens = max_tokens,
    gt = osp.join(DATA_DIR, "queries", "uc_gt.csv"),
    verbose = verbose
)

exp1.run_test_cases(test_cases)

# Save Output
exp1.save_json(save_path+".json")
exp1.write_csv(save_path+".csv")

## Experiment 2: Only Text - CHAT Prompt Template - GPT4

### Prompt 1

In [ ]:
### CHAT PROMTP TEMPLATE
system_prompt = """
Make reference to the context given to assess the scenario. If you do not know the answer. just say that "I don't know", don't try to make up an answer.
You are a physician assistant giving advice on treatment for moderate to severe ulcerative colitis (UC).

ANALYSE the given patient profile based on given query based on one of the following criteria:
- Whether treated patient is new patient or patient under maintenance
- Prior response to Infliximab
- Prior failure to Anti-TNF agents
- Prior failure to Vedolizumab
- Age
- Pregnancy
- Extraintestinale manifestations
- Pouchitis

FINALLY RETURN up to 2 TOP choices of biological drugs given patient profile. Explain the PROS and CONS of the 2 choices.
Output your answer as a list of JSON objects with keys: drug_name, advantages, disadvantages.

{summaries}

"""

TEST_PROMPT_TEMPLATE_2 = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_prompt, input_variables = ["summaries"]),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

print(TEST_PROMPT_TEMPLATE_2.format(summaries = "Summaries", question = "User Query"))

System: 
Make reference to the context given to assess the scenario. If you do not know the answer. just say that "I don't know", don't try to make up an answer.
You are a physician assistant giving advice on treatment for moderate to severe ulcerative colitis (UC).

ANALYSE the given patient profile based on given query based on one of the following criteria:
- Whether treated patient is new patient or patient under maintenance
- Prior response to Infliximab
- Prior failure to Anti-TNF agents
- Prior failure to Vedolizumab
- Age
- Pregnancy
- Extraintestinale manifestations
- Pouchitis

FINALLY RETURN up to 2 TOP choices of biological drugs given patient profile. Explain the PROS and CONS of the 2 choices.
Output your answer as a list of JSON objects with keys: drug_name, advantages, disadvantages.

Summaries


Human: User Query


In [ ]:
# Settings
llm_type = "gpt-4"
description = "Text_Only_With_CHAT_Prompt"
emb_store_dir = os.path.join(EMBSTORE_DIR, "v1_1000_50")
max_tokens = 1024
time = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
verbose = True
save_path = osp.join(ARTIFACT_DIR, f"{llm_type}_{description}_{time}")
print("Save directory:", save_path)

In [ ]:
# Create and run experiment
exp2 = QuestionAnsweringWithIndexSearchExperiment(
    prompt_template = TEST_PROMPT_TEMPLATE_2,
    vector_store = emb_store_dir,
    llm_type = llm_type,
    max_tokens = max_tokens,
    gt = osp.join(DATA_DIR, "queries", "uc_gt.csv"),
    verbose = verbose
)

exp2.run_test_cases(test_cases)

# Save Output
exp2.save_json(save_path+".json")
exp2.write_csv(save_path+".csv")

### Prompt 2

In [28]:
### CHAT PROMTP TEMPLATE
system_prompt = """
Make reference to the context given to assess the scenario. If you do not know the answer. just say that "I don't know", don't try to make up an answer.
You are a physician assistant giving advice on treatment for moderate to severe ulcerative colitis (UC).

Analyse the patient ulcerative colitis (UC) severity and list all risk factors.
Analyse the patient profile and list all risk factors. Patient profile includes age, gender, pregnancy status, prior reactions to any drugs, whether the patient is newly diagnosed, extraintestinale manifestation, pouchtitis

FINALLY RETURN up to 2 TOP choices of recommended biological drugs given patient profile. Explain the PROS and CONS of the 2 choices.
Output your answer as a list of JSON objects with keys: drug_name, advantages, disadvantages.

{summaries}
"""

TEST_PROMPT_TEMPLATE_3 = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_prompt, input_variables = ["summaries"]),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

prompt_str = TEST_PROMPT_TEMPLATE_3.format(summaries = "Summaries", question = "User Query")
print(prompt_str)

System: 
Make reference to the context given to assess the scenario. If you do not know the answer. just say that "I don't know", don't try to make up an answer.
You are a physician assistant giving advice on treatment for moderate to severe ulcerative colitis (UC).

Analyse the patient ulcerative colitis (UC) severity and list all risk factors.
Analyse the patient profile and list all risk factors. Patient profile includes age, gender, pregnancy status, prior reactions to any drugs, whether the patient is newly diagnosed, extraintestinale manifestation, pouchtitis

FINALLY RETURN up to 2 TOP choices of recommended biological drugs given patient profile. Explain the PROS and CONS of the 2 choices.
Output your answer as a list of JSON objects with keys: drug_name, advantages, disadvantages.

Summaries

Human: User Query


In [29]:
# Settings
llm_type = "gpt-4"
description = "CHAT_Prompt_V2"
emb_store = "v5-add-tables_2500_500"
max_tokens = 1024
time = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
verbose = False
save_path = osp.join(ARTIFACT_DIR, f"{llm_type}_{description}_{time}")
print("Save directory:", save_path)

settings = {
    "project": PROJECT,
    "test_case": osp.join(DATA_DIR, "queries", "uc.txt"),
    "prompt": prompt_str,
    "ground_truth": "uc_gt.csv",
    "description": description,
    "verbose": verbose,

    "emb_type": "text-embedding-ada-002",
    "vectorstore": "faiss/text-embedding-ada-002/" + emb_store,
    "chunk_size": emb_store.split("_")[-2],
    "chunk_overlap": emb_store.split("_")[-1],
    "additional_docs": "data/additional_docs.json",
    "pinecone_index_name": None,

    "llm_type": "gpt-4",
    "temperature": 0,
    "max_tokens": max_tokens,
}

Save directory: /mnt/c/Users/QUAN/Desktop/medical-chatbot/artifacts/gpt-4_CHAT_Prompt_V2_07-07-2023-17-34-18


In [ ]:
# Create and run experiment
exp4 = QuestionAnsweringWithIndexSearchExperiment(
    prompt_template = TEST_PROMPT_TEMPLATE_3,
    vector_store = os.path.join(EMBSTORE_DIR, emb_store),
    llm_type = llm_type,
    max_tokens = max_tokens,
    gt = osp.join(DATA_DIR, "queries", "uc_gt.csv"),
    verbose = verbose
)

exp4.run_test_cases(test_cases)

# Save Output
exp4.save_json(os.path.join(save_path+"result.json"))
exp4.write_csv(os.path.join(save_path+"result.csv"))

with open(os.path.join(save_path, "settings.yaml"), "w") as f:
    yaml.dump(settings, f)

## Experiment 3 - Added Table:

In [9]:
### STANDARD PROMPT TEMPLATE
drug_parser = PydanticOutputParser(pydantic_object=DrugOutput)

prompt_template = """Make reference to the context given to assess the scenario. If you do not know the answer. just say that "I don't know", don't try to make up an answer.
You are a physician assistant giving advice on treatment for moderate to severe ulcerative colitis (UC). Perform the following step

ANALYSE the given patient profile based on given query based on one of the following criteria:
- Freshly treated patient or patient under maintenance
- Prior response to Infliximab
- Prior failure to Anti-TNF agents
- Prior failure to Vedolizumab
- Age
- Pregnancy
- Extraintestinale manifestations
- Pouchitis

FINALLY RETURN up to 2 TOP choices of biological drugs given patient profile. Explain the PROS and CONS of the 2 choices.

{summaries}

{format_instructions}

Question: {question}
Answer:
"""

TEST_PROMPT_TEMPLATE_3 = PromptTemplate(
    template = prompt_template,
    input_variables = ["summaries", "question"],
    partial_variables={"format_instructions": drug_parser.get_format_instructions()}
)

print(TEST_PROMPT_TEMPLATE_3.format(summaries = "Summaries", question = "User Query"))

Make reference to the context given to assess the scenario. If you do not know the answer. just say that "I don't know", don't try to make up an answer.
You are a physician assistant giving advice on treatment for moderate to severe ulcerative colitis (UC). Perform the following step

ANALYSE the given patient profile based on given query based on one of the following criteria:
- Freshly treated patient or patient under maintenance
- Prior response to Infliximab
- Prior failure to Anti-TNF agents
- Prior failure to Vedolizumab
- Age
- Pregnancy
- Extraintestinale manifestations
- Pouchitis

FINALLY RETURN up to 2 TOP choices of biological drugs given patient profile. Explain the PROS and CONS of the 2 choices.

Summaries

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
t

In [11]:
# Settings
LLM_TYPE = "gpt-4"
DESCRIPTION = "With Some Tables"
MAX_TOKENS = 1024
TIME = datetime.now().strftime("%d-%m-%Y-%H:%M:%S")
VERBOSE = True
VECTORSTORE = os.path.join(EMBSTORE_DIR, PROJECT, "faiss", "text-embedding-ada-002", "v2-add-rows_1000_200")
save_path = osp.join(ARTIFACT_DIR, f"{LLM_TYPE}_{DESCRIPTION}_{TIME}")

### Run Experiments

In [ ]:
# Create and run experiment
exp3 = QuestionAnsweringWithIndexSearchExperiment(
    prompt_template = TEST_PROMPT_TEMPLATE_3,
    vector_store = VECTORSTORE,
    llm_type = LLM_TYPE,
    max_tokens = MAX_TOKENS,
    gt = osp.join(MAIN_DIR, "ground_truth.csv"),
    verbose = VERBOSE
)

exp3.run_test_cases(test_cases)

# Save Output
exp3.save_json(save_path + ".json")
exp3.write_csv(save_path + ".csv")

## Experiment 4 - Changing to Different Settings

- uc-3 - test_case: data/queries/uc.txt, prompt: uc_qa_source_1.py, embstore: faiss/text-embedding-ada-002/v2-add-rows_1000_200
- uc-4 - test_case: data/queries/uc.txt, prompt: uc_qa_source_1.py, embstore: faiss/text-embedding-ada-002/v3-add-rows_2500_500
- uc-5 - test_case: data/queries/uc.txt, prompt: uc_qa_source_1.py, embstore: faiss/text-embedding-ada-002/v4-add-tables_1000_200
- uc-6 - test_case: data/queries/uc.txt, prompt: uc_qa_source_1.py, embstore: faiss/text-embedding-ada-002/v5-add-tables_2500_500
- uc-7 - test_case: data/queries/uc.txt, prompt: uc_qa_source_1.py, embstore: faiss/text-embedding-ada-002/v6-add-tables_750_100
- uc-8 - test_case: data/queries/uc.txt, prompt: uc_qa_source_2.py, embstore: faiss/text-embedding-ada-002/v5-add-tables_2500_500
- uc-1_chat - test_case: data/queries/uc.txt, prompt: uc_qa_source_chat_1.py, embstore: faiss/text-embedding-ada-002/v5-add-tables_2500_500
- uc-2_chat - test_case: data/queries/uc.txt, prompt: uc_qa_source_chat_1.py, embstore: faiss/text-embedding-ada-002/v3-add-rows_2500_500
- uc-3_chat - test_case: data/queries/uc.txt, prompt: uc_qa_source_chat_1.py, embstore: faiss/text-embedding-ada-002/v4-add-tables_1000_200
- uc-4_chat - test_case: data/queries/uc.txt, prompt: uc_qa_source_chat_2.py, embstore: faiss/text-embedding-ada-002/v5-add-tables_2500_500
- uc-5_chat - test_case: data/queries/uc_long.txt, prompt: uc_qa_source_chat_3.py, embstore: faiss/text-embedding-ada-002/v5-add-tables_2500_500

In [ ]:
! ./src/bash/multi_exps.sh uc_7.yaml uc_8_chat.yaml uc_4_chat.yaml uc_5_chat.yaml

## Experiment 5 - QA from all docs

### Split Documents into Pages/Text Chunks

In [ ]:
documents = load_documents(os.path.join(DOCUMENT_SOURCE, PROJECT),
                           exclude_pages=EXCLUDE_DICT)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000, chunk_overlap=500
)
texts = text_splitter.split_documents(documents)

doc_max_tokens, text_max_tokens = 0, 0

print("Number of text documents:", len(documents))
print("Number of text chunks:", len(texts))

print("Number of tokens in all documents:", check_documents_token(documents))
print("Number of tokens in all text chunks:", check_documents_token(texts))

for document in documents:
    doc_tokens = check_documents_token(document)
    if doc_tokens > doc_max_tokens:
        doc_max_tokens = doc_tokens

for text in texts:
    text_tokens = check_documents_token(text)
    if text_tokens > text_max_tokens:
        text_max_tokens = text_tokens
    
print("Max tokens in all documents:", doc_max_tokens)
print("Max tokens in all text chunks:", text_max_tokens)

Number of text documents: 58
Number of text chunks: 90
Number of tokens in all documents: 83628
Number of tokens in all text chunks: 88263
Max tokens in all documents: 3301
Max tokens in all text chunks: 2211


### MapReduce

#### Setup Prompts & Chains

In [13]:
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo",
                 openai_api_key=OPENAI_KEY)
reduce_llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k",
                        openai_api_key=OPENAI_KEY)
collapse_llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k",
                        openai_api_key=OPENAI_KEY) 

verbose = True

In [44]:
## Map
system_question_template = """You are a physician assistant giving advice on treatment for moderate to severe ulcerative colitis (UC) using biological drugs.
Use the following portion of a long document to see if any of the text is relevant to treatment of given patient profile using biological drugs. 
Return any relevant text verbatim.
______________________
{context}"""
messages = [
    SystemMessagePromptTemplate.from_template(system_question_template),
    HumanMessagePromptTemplate.from_template("Patient Profile: {question}"),
]
CHAT_QUESTION_PROMPT = ChatPromptTemplate.from_messages(messages)

## Reduce

system_combine_template = """You are a physician assistant giving advice on treatment for moderate to severe ulcerative colitis (UC) using biological drugs.
If you don't know the answer, just say that you don't know. Don't try to make up an answer.

Given the following extracted information of a long document, return up to 2 top choices of biological drugs given the patient profile. 
Explain the PROS and CONS of the 2 choices with respect to the patient profile.

Output your answer as a list of JSON objects with keys: drug_name, advantages, disadvantages.
______________________
{summaries}"""
messages = [
    SystemMessagePromptTemplate.from_template(system_combine_template),
    HumanMessagePromptTemplate.from_template("{question}"),
]
CHAT_COMBINE_PROMPT = ChatPromptTemplate.from_messages(messages)

CHAT_COLLAPSE_PROMPT = CHAT_COMBINE_PROMPT

In [15]:
map_chain = LLMChain(llm=llm, prompt=CHAT_QUESTION_PROMPT, verbose=verbose)
reduce_chain = LLMChain(llm=reduce_llm, prompt=CHAT_COMBINE_PROMPT,
                        verbose=verbose)
collapse_chain = LLMChain(llm=collapse_llm, prompt=CHAT_COLLAPSE_PROMPT,
                          verbose=verbose)

combine_document_chain = StuffDocumentsChain(
    llm_chain=reduce_chain,
    document_variable_name="summaries",
    verbose=verbose
    )

collapse_document_chain = StuffDocumentsChain(
    llm_chain=collapse_chain,
    document_variable_name="summaries",
    verbose=verbose
)

map_reduce_qa_chain = MapReduceDocumentsChainV2(
    llm_chain=map_chain,
    combine_document_chain=combine_document_chain,  
    collapse_document_chain=collapse_document_chain,
    document_variable_name="context",
    combine_max_tokens = 14000,
    collapse_max_tokens = 6000,
    verbose=verbose,
    return_map_steps=True
)

#### On Pages

In [ ]:
output_dict = {
    "prompt":{
        "map": BaseExperiment.convert_prompt_to_string(map_chain.prompt),
        "combine": BaseExperiment.convert_prompt_to_string(combine_document_chain.llm_chain.prompt),
        "collapse": BaseExperiment.convert_prompt_to_string(collapse_document_chain.llm_chain.prompt)
    },
    "test_cases": []
}

for test_case in test_cases:
    answer = map_reduce_qa_chain({"input_documents": documents, "question": test_case})
    output_dict["test_cases"].append(
        {
            "question": answer["question"],
            "answer": answer["output_text"],
            "intermediate_steps": answer["intermediate_steps"]
        }
    )
    
with open(os.path.join(ARTIFACT_DIR, "map_reduce_page.json"), "w") as f:
    json.dump(output_dict, f)

#### On Text Chunks

In [55]:
## Add tables as texts
additional_docs = os.path.join(DATA_DIR, "additional_docs.json")

with open(additional_docs, "r") as f:
    add_doc_infos = json.load(f)
for add_doc_info in add_doc_infos:
    if add_doc_info["mode"] == "table":
        extra_docs = convert_csv_to_documents(
            add_doc_info, concatenate_rows=True
            )
        texts.extend(extra_docs)
        documents.extend(extra_docs)
        
    elif add_doc_info["mode"] == "json":
        texts.extend(convert_json_to_documents(add_doc_info))
        documents.extend(convert_json_to_documents(add_doc_info))
    else:
        LOGGER.warning(
            "Invalid document type. No texts added to documents list"
        )

In [49]:
output_dict = {
    "prompt":{
        "map": BaseExperiment.convert_prompt_to_string(map_chain.prompt),
        "combine": BaseExperiment.convert_prompt_to_string(combine_document_chain.llm_chain.prompt),
        "collapse": BaseExperiment.convert_prompt_to_string(collapse_document_chain.llm_chain.prompt)
    },
    "test_cases": []
}

for test_case in test_cases:
    answer = map_reduce_qa_chain({"input_documents": texts, "question": test_case})
    output_dict["test_cases"].append(
        {
            "question": answer["question"],
            "answer": answer["output_text"],
            "intermediate_steps": answer["intermediate_steps"]
        }
    )
    
with open(os.path.join(ARTIFACT_DIR, "map_reduce_chunk_5000_500.json"), "w") as f:
    json.dump(output_dict, f)

### Refine

#### Setup Prompts & Chains

In [ ]:
initial_system_question_template = """You are a physician assistant giving advice on treatment for moderate to severe ulcerative colitis (UC) using biological drugs.
If you don't know the answer, just say that you don't know. Don't try to make up an answer.

Given the below context information and no prior knowledge, return up to 2 top choices of biological drugs given the patient profile.
Explain the PROS and CONS of the 2 choices with respect to the patient profile.

Output your answer as a list of JSON objects with keys: drug_name, advantages, disadvantages.

---------------------
{context_str}
---------------------
"""

init_messages = [
    SystemMessagePromptTemplate.from_template(initial_system_question_template),
    HumanMessagePromptTemplate.from_template("Patient Profile: {question}"),
]
CHAT_INITIAL_PROMPT = ChatPromptTemplate.from_messages(init_messages)

refine_system_question_template = """You are a physician assistant giving advice on treatment for moderate to severe ulcerative colitis (UC) using biological drugs.
If you don't know the answer, just say that you don't know. Don't try to make up an answer.

Return up to 2 top choices of biological drugs given the patient profile.
Explain the PROS and CONS of the 2 choices with respect to the patient profile.

Output your answer as a list of JSON objects with keys: drug_name, advantages, disadvantages.
"""

refine_template = (
    "We have the opportunity to refine the existing answer"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{context_str}\n"
    "------------\n"
    "Given the new context, refine the original answer to better "
    "answer the question. "
    "If the context isn't useful, return the original answer."
)

refine_template = """
We have the opportunity to refine the existing answer (only if needed) with some more context below.

---------------------
{context_str}
---------------------

Given the new context, refine the original answer to better answer the question. If the context isn't useful, return the original answer.
Output your answer as a list of JSON objects with keys: drug_name, advantages, disadvantages.
"""

refine_messages = [
    SystemMessagePromptTemplate.from_template(refine_system_question_template),
    HumanMessagePromptTemplate.from_template("Patient Profile: {question}"),
    AIMessagePromptTemplate.from_template("Existing answer: {existing_answer}"),
    HumanMessagePromptTemplate.from_template(refine_template),
]
CHAT_REFINE_PROMPT = ChatPromptTemplate.from_messages(refine_messages

In [ ]:
verbose = True

init_llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo",
                 openai_api_key=OPENAI_KEY)
refine_llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo",
                        openai_api_key=OPENAI_KEY)

initial_chain = LLMChain(llm=llm, prompt=CHAT_INITIAL_PROMPT,verbose=verbose)
refine_chain=LLMChain(llm=refine_llm, prompt=CHAT_REFINE_PROMPT, verbose=verbose)

refine_qa_chain = RefineDocumentsChain(
    initial_llm_chain=initial_chain,
    refine_llm_chain=refine_chain,
    document_variable_name="context_str",
    initial_response_name="existing_answer",
    return_intermediate_steps=True,
    verbose=verbose,
)

In [ ]:
### On Pages + Extra Documents

output_dict = {
    "prompt":{
        "initial": BaseExperiment.convert_prompt_to_string(CHAT_INITIAL_PROMPT),
        "refine": BaseExperiment.convert_prompt_to_string(CHAT_REFINE_PROMPT)
    },
    "test_cases": []
}

for test_case in test_cases:
    answer = map_reduce_qa_chain({"input_documents": texts, "question": test_case})
    output_dict["test_cases"].append(
        {
            "question": answer["question"],
            "answer": answer["output_text"],
            "intermediate_steps": answer["intermediate_steps"]
        }
    )
    
with open(os.path.join(ARTIFACT_DIR, "map_reduce_chunk_5000_500.json"), "w") as f:
    json.dump(output_dict, f)

## Experiment 6: Problem breakdown + Semantic Search

### Identify individual patient features 

In [ ]:
class NewLineSeparatedListOutputParser(ListOutputParser):
    """Parse out comma separated lists."""

    def get_format_instructions(self) -> str:
        return (
            "Your response should be a list of newline separated values, "
            "eg: `foo\nbar\nbaz`"
        )

    def parse(self, text: str) -> List[str]:
        """Parse the output of an LLM call."""
        return text.strip().split("\n")

In [42]:
breakdown_system_prompt = """You are a physician assistant giving advice on treatment for moderate to severe ulcerative colitis (UC).
If you do not know the answer. just say that "I don't know", don't try to make up an answer.

Analyse the patient profile and medical history and list all related risk factors.
- Newly treated patient or patient under maintenance
- Prior response/failure to biological drugs (Infliximab, Anti-TNF agents, Vedolizumab, etc)
- Age
- Pregnancy Status
- Extraintestinale manifestations
- Pouchitis

Return your answer as a list of risk factors.
-------------------------
Example 1
Human:
Patient Profile: 36 year old woman with moderate ulcerative colitis and multiple sclerosis

AI: 
- Age: 36 years old female
- Medical history: Having multiple sclerosis.

Example 2
Human: 
Patient Profile: 40 year old pregnant female with newly diagnosed moderate UC and articular extraintestinal manifestations

AI: 
- Age: 40 years old
- Pregnant female
- Newly diagnosed 
- Medical history: Having multiple sclerosis.

Example 3:
Human:
Patient Profile: 35 year old male with known moderate UC with prior exposure to infliximab but has worsening colitis on endoscopy despite compliance

AI:
- Age: 35 years old male
- Medical history: Patient under maintenance treatment of Infliximab with worsening condition despite medication.
"""

breakdown_messages = [
    SystemMessagePromptTemplate.from_template(breakdown_system_prompt),
    HumanMessagePromptTemplate.from_template("Patient Profile: {question}")        
]

BREAKDOWN_PROMPT = ChatPromptTemplate.from_messages(
    messages=breakdown_messages,
    output_parser=NewLineSeparatedListOutputParser()
    )

breakdown_llm = ChatOpenAI(
    model_name="gpt-4",
    max_tokens=1024,
    openai_api_key=OPENAI_KEY
)

chain = LLMChain(
    prompt=BREAKDOWN_PROMPT,
    llm=breakdown_llm,
    output_key="text_output"
)

In [46]:
test_results = {
    "queries": test_cases,
    "breakdown_output": [],
}

for test_case in test_cases:
    output = chain({"question": test_case})
    test_results["features_list"].append(output["text_output"])

### Run QA on each feature

In [67]:
feature_prompt = """You are a physician assistant giving advice on treatment for moderate to severe ulcerative colitis (UC). Make reference to the REFERENCE TEXT given to assess the scenario.

Based on the patient profile or medical history, make recommendations of biological drugs for UC treatment. If there are no suitable drugs. just say that "I don't know", don't try to make up an answer.
For each recommended drug, explain the PROS and CONS in context of the patient profile.
Also return drugs which this patient should avoid given his profile or medical history and reasons why the drug should be avoided.

=========
REFERENCE TEXT:
{summaries}
"""

human_prompt = """
=========
PATIENT RISK PROFILE: {question}
=========
"""

FEATURE_PROMPT_TEMPLATE = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            feature_prompt, input_variables=["summaries"]
        ),
        HumanMessagePromptTemplate.from_template(human_prompt),
    ]
)

verbose = True

qa_llm=ChatOpenAI(
    model_name="gpt-4", temperature=0, max_tokens=1024,
    openai_api_key=OPENAI_KEY, verbose=verbose
    )

vectorstore = os.path.join(EMBSTORE_DIR, "v8-add-tables_2500_500")
docsearch = FAISS.load_local(vectorstore, OpenAIEmbeddings(openai_api_key=OPENAI_KEY))

qa_from_docs_chain=RetrievalQAWithSourcesChain.from_chain_type(
    llm=qa_llm,
    chain_type="stuff",
    chain_type_kwargs={"prompt": FEATURE_PROMPT_TEMPLATE},
    retriever=docsearch.as_retriever(search_kwargs={"k": 10}),
    return_source_documents=True,
    max_tokens_limit=6500,
    reduce_k_below_max_tokens=True,
    verbose=verbose
)

In [ ]:
output_list = []
for intermediate_answers in test_results["features_list"]:
    output_dict = {}
    for intermediate_answer in intermediate_answers:
        output = qa_from_docs_chain(intermediate_answer)
        output_dict[output["question"]] = output["answer"]
    output_list.append(output_dict)

In [74]:
with open(os.path.join(ARTIFACT_DIR, "intermediate_answers.json"), "w") as f:
    json.dump(output_list, f)

### Combine feature answers into final answer

In [ ]:
combine_prompt = """You are a physician assistant giving advice on treatment for moderate to severe ulcerative colitis (UC). If you do not know the answer. just say that "I don't know", don't try to make up an answer.
Make reference to the REFERENCE ANSWERS given to assess the scenario.

Combine the answers in REFERENCE ANSWERS to recommend up to 2 TOP choices of biological drugs given patient profile. Explain the PROS and CONS of the choices recommended.
Prioritize drugs recommendations based on patients medical conditions and history.
Output your answer as a list of JSON objects with keys: drug_name, advantages, disadvantages.
=========
REFERENCE ANSWERS:
{summaries}

=========
COMBINED ANSWER:
"""

COMBINE_PROMPT_TEMPLATE = PromptTemplate.from_template(
    template=combine_prompt

combine_llm=ChatOpenAI(
    model_name="gpt-4",
    temperature=0,
    max_tokens=1024,
    openai_api_key=OPENAI_KEY
)

combine_chain = LLMChain(
    llm=combine_llm,
    prompt=COMBINE_PROMPT_TEMPLATE,
    output_key="output_text"
)

In [86]:
summaries = []
for intermediate_answers in output_list:
    summary = ""
    for feature, answer in intermediate_answers.items():
        summary += f"Patient Feature: {feature}\nAnswer: {answer}\n------------------------\n"
    summaries.append(summary)

In [91]:
final_answers = []
for summary in summaries:
    output = combine_chain({"summaries": summary})
    final_answers.append(output["output_text"])

In [92]:
with open(os.path.join(ARTIFACT_DIR, "final_answers.json"), "w") as f:
    json.dump(final_answers, f)

### Write Output Answers to file

In [96]:
exp6 = QuestionAnsweringOverDocsExperiment(
    gt = os.path.join(DATA_DIR, "queries", "uc_gt.csv")
)

In [98]:
exp6.questions = test_cases
exp6.answers = final_answers
exp6.intermediate_steps = output_list
exp6.prompt_map = {
    "breakdown": BREAKDOWN_PROMPT,
    "feature": FEATURE_PROMPT_TEMPLATE,
    "combine": COMBINE_PROMPT_TEMPLATE
}

In [99]:
exp6.save_json(os.path.join(ARTIFACT_DIR, "output.json"))
exp6.write_csv(os.path.join(ARTIFACT_DIR, "output.csv"))

## Experiment 7: QA over Summaries

### Prepare Documents

In [16]:
documents = load_documents(os.path.join(DOCUMENT_SOURCE, PROJECT),
                           exclude_pages=EXCLUDE_DICT)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000, chunk_overlap=500
)
texts = text_splitter.split_documents(documents)

doc_max_tokens, text_max_tokens = 0, 0

print("Number of text documents:", len(documents))
print("Number of text chunks:", len(texts))

## Add tables as texts
additional_docs = os.path.join(DATA_DIR, "additional_docs.json")

with open(additional_docs, "r") as f:
    add_doc_infos = json.load(f)
for add_doc_info in add_doc_infos:
    if add_doc_info["mode"] == "table":
        extra_docs = convert_csv_to_documents(
            add_doc_info, concatenate_rows=True
            )
        texts.extend(extra_docs)
        documents.extend(extra_docs)
        
    elif add_doc_info["mode"] == "json":
        texts.extend(convert_json_to_documents(add_doc_info))
        documents.extend(convert_json_to_documents(add_doc_info))
    else:
        print.warning(
            "Invalid document type. No texts added to documents list"
        )
        
print("Number of tokens in all documents:", check_documents_token(documents))
print("Number of tokens in all text chunks:", check_documents_token(texts))

Number of text documents: 58
Number of text chunks: 90
Number of tokens in all documents: 84929
Number of tokens in all text chunks: 89564


In [43]:
def create_summarize_chain(
    summarize_prompt: ChatPromptTemplate,
    map_llm_type: str = "gpt-3.5-turbo",
    reduce_llm_type: str = "gpt-3.5-turbo-16k",
    combine_max_doc_tokens: int = 14000,
    collapse_max_doc_tokens: int = 6500,
    map_tokens_limit: int = 512,
    reduce_tokens_limit: int = 1024,
    return_intermediate_steps: bool = True,
    verbose: bool = False
):

    ## LLM & Chains
    map_llm = ChatOpenAI(model_name=map_llm_type, temperature=0,
                    max_tokens=map_tokens_limit, openai_api_key=OPENAI_KEY)
    map_chain = LLMChain(llm=map_llm, prompt=summarize_prompt, verbose=verbose)

    reduce_llm = ChatOpenAI(model_name=reduce_llm_type, temperature=0,
                        max_tokens=reduce_tokens_limit, openai_api_key=OPENAI_KEY)
    reduce_chain = LLMChain(llm=reduce_llm, prompt=summarize_prompt, verbose=verbose)
    combine_document_chain = StuffDocumentsChain(
        llm_chain=reduce_chain, document_variable_name="text", verbose=verbose
    )

    collapse_llm = ChatOpenAI(model_name=reduce_llm_type, temperature=0,
                        max_tokens=reduce_tokens_limit, openai_api_key=OPENAI_KEY)
    collapse_chain = LLMChain(llm=collapse_llm, prompt=summarize_prompt, verbose=verbose)

    collapse_document_chain = StuffDocumentsChain(
        llm_chain=collapse_chain, document_variable_name="text", verbose=verbose
    )

    return MapReduceDocumentsChainV2(
        llm_chain=map_chain,
        combine_document_chain=combine_document_chain,
        collapse_document_chain=collapse_document_chain,
        document_variable_name="text",
        combine_max_tokens=combine_max_doc_tokens,
        collapse_max_tokens=collapse_max_doc_tokens,
        return_intermediate_steps=return_intermediate_steps,
        return_map_steps=return_intermediate_steps
        )

### Summary: AGE

In [44]:
summarize_system_prompt = """You are a physician assistant giving advice on treatment for moderate to severe ulcerative colitis (UC).
Your job is to extract relevant from REFERENCE TEXT and write a summary based on specified requirements.
"""

age_summarize_human_prompt = """
Use the following REFERENCE TEXT to write a summary on recommended drugs treatment for patients based on different age group.
For each age group, return the recommended drugs and drugs to avoid, together with the PROS and CONS.
If there are no recommendations based on age, just say "No relevant Information", do not make up an answer.

=========
REFERENCE TEXT:
"{text}"
=========
"""

messages = [
    SystemMessagePromptTemplate.from_template(summarize_system_prompt),
    HumanMessagePromptTemplate.from_template(age_summarize_human_prompt)
]

AGE_SUMMARIZE_PROMPT = ChatPromptTemplate.from_messages(
    messages=messages
)

In [45]:
age_summarize_chain = create_summarize_chain(summarize_prompt = AGE_SUMMARIZE_PROMPT)
age_summary = age_summarize_chain({"input_documents": documents})

print(age_summary["output_text"])

Based on the provided reference text, there is no specific information on recommended drugs for different age groups in the treatment of moderate to severe ulcerative colitis (UC). The text primarily focuses on comparing the efficacy and safety of various agents for induction therapy in biologic-naïve patients and those with prior exposure to tumor necrosis factor (TNF) antagonists. In biologic-naïve patients, infliximab was ranked highest for induction of remission and endoscopic improvement. In patients with prior exposure to TNF antagonists, ustekinumab and tofacitinib were ranked highest. However, the text does not provide any recommendations based on age groups. Therefore, no relevant information is available regarding recommended drugs and drugs to avoid for different age groups in the treatment of moderate to severe UC.


### Summary: PREGNANCY STATUS

In [46]:
summarize_system_prompt = """You are a physician assistant giving advice on treatment for moderate to severe ulcerative colitis (UC).
Your job is to extract relevant from REFERENCE TEXT and write a summary based on specified requirements.
"""

pregnancy_summarize_human_prompt = """
Use the following REFERENCE TEXT to write a summary on recommended drugs treatment for patients based on pregnancy status.
In case patient is pregnant, return the recommended drugs and drugs to avoid, together with the PROS and CONS.
If there are no recommendations based on pregnancy, just say "No relevant Information", do not make up an answer.

=========
REFERENCE TEXT:
"{text}"
=========
"""

messages = [
    SystemMessagePromptTemplate.from_template(summarize_system_prompt),
    HumanMessagePromptTemplate.from_template(pregnancy_summarize_human_prompt)
]

PREGNANCY_SUMMARIZE_PROMPT = ChatPromptTemplate.from_messages(
    messages=messages
)

In [47]:
pregnancy_summarize_chain = create_summarize_chain(summarize_prompt = PREGNANCY_SUMMARIZE_PROMPT)
pregnancy_summary = pregnancy_summarize_chain({"input_documents": documents})

print(pregnancy_summary["output_text"])

No relevant information is provided in the reference text regarding the recommended drugs treatment for patients with moderate to severe ulcerative colitis based on pregnancy status.


# Agents

## Experiment 8: Custom Agent

### CSV Agent

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

model_name = 'text-davinci-003'
temperature = 0.0
model = OpenAI(model_name=model_name, temperature=temperature, openai_api_key = OPENAI_KEY)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

joke_query = "Tell me a joke."
_input = prompt.format_prompt(query=joke_query)

output = model(_input.to_string())